# Experimenting with some code to calculate the robot position relative to the gear target using the image data from the Raspberry Pi

In [220]:
import math

## Use the tape height to determine the distance to a tape

In [219]:
# Knowns
tape_height_inches = 5.0
camera_field_of_view = 70.0 # degrees - sort of the angle subtended
camera_width_pixels = 1280.
camera_height_pixels = 800.

In [154]:
# Measured
tape_height_pixels = 200.

In [155]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print __version__ # requires version >= 1.9.0


2.0.2


In [156]:
init_notebook_mode(connected=True)

In [157]:
#iplot([{"x": [1, 2, 3], "y": [3, 1, 6]}])

In [158]:
# Compute tape distance from camera 
#  using these equations
#   tan(tape_height_angle_subtended) = tape_height_inches / tape_distance_from_camera
#   tape_height_angle_subtended = camera_field_of_view * tape_height_pixels / camera_height_pixels
#
# tape_distance_from_camera = tape_height_inches / tan(tape_height_angle_subtended)
#
tape_height_angle_subtended = camera_field_of_view * tape_height_pixels / camera_height_pixels
tape_distance_from_camera = tape_height_inches / math.tan(math.radians(tape_height_angle_subtended))

In [159]:
tape_distance_from_camera

15.857974011816063

In [160]:
def calc_distance_from_camera_from_heights( pixel_height, height_inches):
    height_angle_subtended = camera_field_of_view * pixel_height / camera_height_pixels
    distance_from_camera = height_inches / math.tan(math.radians(height_angle_subtended))
    return distance_from_camera

In [161]:
calc_distance_from_camera_from_heights(tape_height_pixels, tape_height_inches)

15.857974011816063

## For simple case of robot pointing at target but at an angle, compute distance to target

In [162]:
# Knowns
tape_width_inches = 2.
tape_centers_delta_inches = 8.25

In [163]:
# for this scenario
tilt_angle = 20. # degrees
target_distance_from_camera = 30. 

In [164]:
tape_right_distance_from_camera = target_distance_from_camera + (tape_centers_delta_inches/2.0) * math.tan(math.radians(tilt_angle))
tape_left_distance_from_camera = target_distance_from_camera - (tape_centers_delta_inches/2.0) * math.tan(math.radians(tilt_angle))

In [165]:
tape_right_distance_from_camera

31.501377216348086

In [166]:
tape_left_distance_from_camera

28.498622783651914

#### first see what is the difference in pixel height for this scenario

In [167]:
def calc_pixel_height_from_distance_and_actual_height(distance_from_camera,height_inches):
    angle_subtended =  math.degrees(math.atan(height_inches/distance_from_camera))
    pixel_height = ( camera_height_pixels / camera_field_of_view ) * angle_subtended
    return pixel_height

In [168]:
# What are the heights in pixels for the two pieces of tape?
tape_right_height_pixels = ( camera_height_pixels / camera_field_of_view ) * math.degrees(math.atan(tape_height_inches/tape_right_distance_from_camera))
tape_left_height_pixels = ( camera_height_pixels / camera_field_of_view ) * math.degrees(math.atan(tape_height_inches/tape_left_distance_from_camera))

In [169]:
tape_right_height_pixels

103.07353822816032

In [170]:
tape_left_height_pixels

113.72683368382303

#### now go the other way. Given the pixel heights, what is the computed tilt_angle?

In [171]:
computed_tape_right_distance_from_camera = calc_distance_from_camera_from_heights(tape_right_height_pixels,tape_height_inches)
computed_tape_left_distance_from_camera = calc_distance_from_camera_from_heights(tape_left_height_pixels,tape_height_inches)

In [172]:
computed_tape_right_distance_from_camera

31.50137721634808

In [173]:
computed_tape_left_distance_from_camera

28.49862278365191

In [174]:
pixel_height = calc_pixel_height_from_distance_and_actual_height(computed_tape_right_distance_from_camera,tape_height_inches)
pixel_height

103.07353822816034

In [175]:
def triangle_angle (a, b, c):
    return math.degrees(math.acos((c**2 - b**2 - a**2)/(-2.0 * a * b)))

In [176]:
triangle_angle(5.,3.,4.)

53.13010235415599

In [177]:
# Not really correct! Just looks the same but totally unrelated to what I want to calculate
triangle_angle(computed_tape_left_distance_from_camera,computed_tape_right_distance_from_camera,tape_centers_delta_inches)

14.734603526159638

## Use the x pixel value for the center of the tape and the distance to the tape to compute the (x,y) coordinates of the center of the tape

In [178]:
# Measured
#   x_center_right_tape_pixels

In [179]:
x_center_right_tape_pixels = 150.0
x_center_left_tape_pixels = - 250.0

In [180]:
# tan of angle from axis of camera to angle that intersects the center of the tape = 
#      x coordinate of tape center in the coordinates of the camera
#         divided by
#      distance to the tape center which we can get from the calculations above using the tape height in pixels
# the angle is approximately equal to       
#         x coordinate of the center of the tape in pixels ( 0 in the middle of the camera
#              times
#         camera_field_of_view
#              divided by 
#         camera_width_pixels
#
#   Using those two equations, you can compute 
#         x coordinate of tape center in the coordinates of the camera

In [181]:
x_center_right_tape = computed_tape_right_distance_from_camera * math.sin(math.radians(x_center_right_tape_pixels * camera_field_of_view / camera_width_pixels))
y_center_right_tape = computed_tape_right_distance_from_camera * math.cos(math.radians(x_center_right_tape_pixels * camera_field_of_view / camera_width_pixels))
x_center_right_tape, y_center_right_tape, computed_tape_right_distance_from_camera

(4.494708403735885, 31.179069307662793, 31.50137721634808)

In [182]:
x_center_left_tape = computed_tape_left_distance_from_camera * math.cos(math.radians(x_center_left_tape_pixels * camera_field_of_view / camera_width_pixels))
x_center_left_tape

27.691121733682024

In [183]:
def calc_xy_tape_center(computed_tape_distance_from_camera, x_center_tape_pixels):
    x_center_tape = computed_tape_distance_from_camera * math.sin(math.radians(x_center_tape_pixels * camera_field_of_view / camera_width_pixels))
    y_center_tape = computed_tape_distance_from_camera * math.cos(math.radians(x_center_tape_pixels * camera_field_of_view / camera_width_pixels))
    return x_center_tape, y_center_tape

In [184]:
x_center_right_tape, y_center_right_tape = calc_xy_tape_center(computed_tape_right_distance_from_camera, x_center_right_tape_pixels)
x_center_right_tape, y_center_right_tape

(4.494708403735885, 31.179069307662793)

In [185]:
x_center_left_tape, y_center_left_tape = calc_xy_tape_center(computed_tape_left_distance_from_camera, x_center_left_tape_pixels)
x_center_left_tape, y_center_left_tape

(-6.735968949994258, 27.691121733682024)

In [186]:
def calc_xy_target(x_center_right_tape, y_center_right_tape,x_center_left_tape, y_center_left_tape ):
    x_target_center = (x_center_right_tape + x_center_left_tape) * 0.5
    y_target_center = (y_center_right_tape + y_center_left_tape) * 0.5
    return x_target_center, y_target_center

In [187]:
x_target_center, y_target_center = calc_xy_target(x_center_right_tape, y_center_right_tape,x_center_left_tape, y_center_left_tape)
x_target_center, y_target_center

(-1.1206302731291866, 29.43509552067241)

## Full test: 
    - set locations of the tape in camera coordinates
    - compute the resulting values in the pixel coordinates of the camera image
    - reverse that going from the pixel coordinates to the locations of the tape

The final function has these as inputs:
    - x_tape_left_center_pixels - coordinate system has 0,0 in the middle
    - x_tape_left_height_pixels
    - y_tape_right_center_pixels
    - y_tape_right_height_pixels
    
And these as its outputs:
    - x_target
    - y_target
    - angle_from_parallel

In [217]:
# For this scenario, choose these values
distance = 24.0
x_tape_left_center = 0.0
y_tape_left_center = distance
x_tape_right_center = 8.25 * math.sin(math.radians(45.0))
y_tape_right_center = distance + 8.25 * math.cos(math.radians(45.0))
x_tape_right_center, y_tape_right_center

(5.833630944789016, 29.833630944789018)

In [208]:
def calc_x_pixel_from_xy(x,y):
    '''x,y is in the camera coordinates'''
    angle = math.degrees(math.atan(x/y))
    print(angle)
    return ( camera_width_pixels / camera_field_of_view ) * angle

In [212]:
# left tape
x_tape_left_center_pixels = calc_x_pixel_from_xy(x_tape_left_center, y_tape_left_center)
tape_left_height_pixels = calc_pixel_height_from_distance_and_actual_height(distance,tape_height_inches)

# right tape
x_tape_right_center_pixels = calc_x_pixel_from_xy(x_tape_right_center, y_tape_right_center)
tape_right_distance = math.sqrt(x_tape_right_center**2 + y_tape_right_center**2)
tape_right_height_pixels = calc_pixel_height_from_distance_and_actual_height(tape_right_distance,tape_height_inches)

x_tape_left_center_pixels, tape_left_height_pixels, x_tape_right_center_pixels, tape_right_height_pixels

0.0
11.0639434076


(0.0, 134.49473065166453, 202.3121080248823, 106.74787765397747)

In [215]:
def calc_target_location(x_tape_left_center_pixels, tape_left_height_pixels, x_tape_right_center_pixels, tape_right_height_pixels):
    tape_left_distance = calc_distance_from_camera_from_heights( tape_left_height_pixels, tape_height_inches)
    tape_right_distance = calc_distance_from_camera_from_heights( tape_right_height_pixels, tape_height_inches)
    
    x_center_left_tape, y_center_left_tape = calc_xy_tape_center(tape_left_distance, x_tape_left_center_pixels)
    x_center_right_tape, y_center_right_tape = calc_xy_tape_center(tape_right_distance, x_tape_right_center_pixels)
    
    return x_center_left_tape, y_center_left_tape, x_center_right_tape, y_center_right_tape

In [216]:
x_center_left_tape, y_center_left_tape, x_center_right_tape, y_center_right_tape = calc_target_location(x_tape_left_center_pixels, tape_left_height_pixels, x_tape_right_center_pixels, tape_right_height_pixels)
x_center_left_tape, y_center_left_tape, x_center_right_tape, y_center_right_tape

(0.0, 24.0, 5.833630944789015, 29.833630944789014)